# Playing with generator

In [1]:
#%%timeit
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import numpy as np
import sys
import copy
from pathlib import Path
#from src.envs.generators import RandomRecurrence
#from src.envs.encoders import Equation, IntegerSeries, RealSeries
from sympy import *
import pickle
from collections import defaultdict, OrderedDict
import math

%reload_ext autoreload

#%cd ~/recur/notebooks
sys.path.append('/private/home/pakamienny/Research_2/recur')
import src
from src.envs.generators import RandomRecurrence
from src.envs.encoders import Equation, RealSeries, IntegerSeries
from src.envs.recurrence import RecurrenceEnvironment


In [27]:
class params:
    float_precision = 3
    max_len = 30
    max_degree = 6
    max_number = 1e100
    real_series = True
    output_numeric=False
    max_exponent = 100
    init_scale = 10
    prob_const=1/3
    prob_n = 1/3
    max_int = 10
    prob_rand = 0.05
    dimension=1
    int_base=10000
    n_predictions=5
    float_tolerance=0.1
    more_tolerance="0.1"
    max_ops=6
    max_token_len=200
    eval_noise=False
    
generator = RandomRecurrence(params)
env = RecurrenceEnvironment(params)
env.rng = np.random.RandomState(0)
import time 

SPECIAL_WORDS = ["EOS", "PAD", "(", ")", "SPECIAL"]

if params.real_series:
    input_encoder = RealSeries(params)
else:
    input_encoder = IntegerSeries(params)
input_words = SPECIAL_WORDS+sorted(list(set(input_encoder.symbols)))

if params.output_numeric:
    if params.real_series:
        output_encoder = RealSeries(params)
    else:
        output_encoder = IntegerSeries(params)
else:
    output_encoder = Equation(params)

deb = time.time()
k_trials=100000
successful_trials=0
size_input=[]
size_output=[]
n_ops_stats = {i:0 for i in range(1,(params.max_ops+1)*params.dimension)}
for k in range(k_trials):
    x, y, tree = env.gen_expr(False)
    
    if tree is not None:
    
        successful_trials+=1
        size_input.append(len(x))
        size_output.append(len(y))
        n_ops = env.code_class(tree)
        n_ops_stats[n_ops]+=1
        #print(series)
        #for i in range(params.dimension):
        #    plt.title(tree.infix())
        #    plt.plot(series[i::params.dimension])
        #plt.yscale('symlog')
        #plt.show()
size_input=np.array(size_input)
size_output=np.array(size_output)
n_valid_equations = sum(list(n_ops_stats.values()))
for n_ops in n_ops_stats:
    n_ops_stats[n_ops]=round(n_ops_stats[n_ops]/n_valid_equations,2)
print("Percentage of successful generation: {}".format(successful_trials/k_trials*100))
print("Time: {}".format(time.time()-deb))
print("Average sizes: input {}, output {}".format(size_input.max(), size_output.max()))
print(n_ops_stats)

/private/home/pakamienny/Research_2/recur/src/envs/generators.py:119: RuntimeWarning: invalid value encountered in double_scalars
  return self.children[0].val(series) - self.children[1].val(series)


Percentage of successful generation: 54.458
Time: 103.8815348148346
Average sizes: input 90, output 13
{1: 0.15, 2: 0.17, 3: 0.18, 4: 0.17, 5: 0.17, 6: 0.16}


# Checking encoder

In [3]:
class params:
    int_base = 10
    has_separator = False
    
encoder = Equation(params)
print(tree.infix())
encoded = encoder.encode(tree)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded.infix())

AttributeError: 'NoneType' object has no attribute 'infix'

In [ ]:
class params:
    a=1
    
from src.envs.encoders import RealSeries

encoder = RealSeries(params)
x = series
print(x)
encoded = encoder.encode(x)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded)